In [2]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import datetime
import time

def crawl_price(date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c):None for c in ' '})for i in r.text.split('\n') if len(i.split('",')) == 17 and i[0]!= '='])),header=0)
    ret = ret.set_index('證券代號')
    ret['成交金額']= ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    return ret

data = {}
n_days = 9
date = datetime.datetime.now()
fail_cnt =0
allow_continuous_fail_cnt = 5
while len(data) < n_days:
    print('parsing', date)
    try:
        data[date.date()] = crawl_price(date)
        print('success!')
        fail_cnt =0
    except:
        print('fail.check the date is holiday')
        fail_cnt +=1
        if fail_cnt == allow_continuous_fail_cnt:
            raise
            break

    date -= datetime.timedelta(days=1)
    time.sleep(10)
close = pd.DataFrame({k:d['收盤價'] for k,d in data.items()}).transpose()
close.index = pd.to_datetime(close.index)
close


parsing 2021-03-29 17:00:53.074478
fail.check the date is holiday
parsing 2021-03-28 17:00:53.074478
fail.check the date is holiday
parsing 2021-03-27 17:00:53.074478
fail.check the date is holiday
parsing 2021-03-26 17:00:53.074478
fail.check the date is holiday
parsing 2021-03-25 17:00:53.074478
fail.check the date is holiday


ConnectionError: HTTPConnectionPool(host='www.twse.com.tw', port=80): Max retries exceeded with url: /exchangeReport/MI_INDEX?response=csv&date=20210325&type=ALL (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E4F59670F0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [ ]:
print(date[datetime.date(2021,3,26)])